In [1]:
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
AP = 'may_AP.csv'
ML = 'may_ML.csv'

In [3]:
aprioriDF = pd.read_csv(f'../data/monthwise/{AP}', index_col=0)
item_name   = pd.read_csv('../data/itemlist.csv', index_col=0)
mlDF = pd.read_csv(f"../data/monthwise/{ML}")

In [4]:
item_name   = item_name[(item_name['DEPID'] != 65) & (item_name['DEPID'] != 21)]
item_name   = item_name[['SKU','KEYWORD']]
df = aprioriDF.merge(item_name, on='SKU')
basket = (df.groupby(['TXNID', 'SKU'])['QUANTITY'].sum().unstack().reset_index().fillna(0).set_index('TXNID'))
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
frequent_itemsets = apriori(basket_sets, min_support=0.00035, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules = rules[rules['confidence'] >= 0.5].sort_values('lift', ascending=False)
rules['antecedent support'] = rules['antecedent support']*len(df)
rules['consequent support'] = rules['consequent support']*len(df)
rules['support'] = rules['support']*len(df)

support=[]
uniquedict = {}
for x in rules.iterrows():
    if x[1]['support'] not in support:
        support.append(x[1]['support'])
        uniquedict[x[1]['support']]=x[1]['antecedents'].union(x[1]['consequents'])
    else:
        uniquedict[x[1]['support']] = uniquedict[x[1]['support']].union(x[1]['antecedents']).union(x[1]['consequents'])
        

In [5]:
count = 0
countlist = []
item=[]
for x in uniquedict:
    count+=1
    for i in uniquedict[x]:
        countlist.append(count)
        item.append([i][0])
basketlist = pd.DataFrame({"GROUP":countlist, "SKU":item})
basketlist = basketlist[['GROUP', 'SKU']]

In [6]:
quant = mlDF[['SKU','QUANTITY']]
kmeans = KMeans(n_clusters=5)
kmeans.fit(quant[['QUANTITY']])
quant['Q_Label'] = kmeans.predict(quant[['QUANTITY']])
label = ['1','2','3','4','5']
quantLabel = quant.groupby(['Q_Label']).agg({'QUANTITY': 'mean'}).reset_index().sort_values("QUANTITY")
quantLabel['QUANTITY_L']=label

cost = mlDF[['SKU','COST']]
kmeans = KMeans(n_clusters=5)
kmeans.fit(cost[['COST']])
cost['C_Label'] = kmeans.predict(cost[['COST']])
label = ['1','2','3','4','5']
costLabel = cost.groupby(['C_Label']).agg({'COST': 'mean'}).reset_index().sort_values("COST")
costLabel['COST_L'] = label

mar = mlDF[['SKU','MARGIN']]
kmeans = KMeans(n_clusters=5)
kmeans.fit(mar[['MARGIN']])
mar['M_Label'] = kmeans.predict(mar[['MARGIN']])
label = ['1','2','3','4','5']
marLabel = mar.groupby(['M_Label']).agg({'MARGIN': 'mean'}).reset_index().sort_values("MARGIN")
marLabel['MARGIN_L'] = label

/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

In [7]:
FinalDF = basketlist.merge(quant, on='SKU').merge(quantLabel, on='Q_Label').merge(cost, on='SKU').merge(costLabel, on='C_Label').merge(mar, on='SKU').merge(marLabel, on='M_Label').merge(item_name, on='SKU')


In [8]:
FinalDF = FinalDF[['GROUP','KEYWORD','COST_L','MARGIN_L','QUANTITY_L']].sort_values('GROUP')
FinalDF

,GROUP,KEYWORD,COST_L,MARGIN_L,QUANTITY_L
0,1,KEG DEPOSIT,2,1,2
4,1,TAP DEPOSIT,3,1,1
6,2,BLACKSTONE CAB SAUV 1.5 LT,1,3,2
7,2,BLACKSTONE CALIFORNIA CABERNET 750 ML,1,2,2
1,3,LUNETTA PROSECCO 750 ML,1,1,2
2,3,MARLBORO SILVER,1,1,2
3,4,MERIT BLUE,1,1,2
5,4,JACK DANIELS #7 BLACK 1.75 LT,3,1,3
8,4,SEAGRAMS LIME TWISTED GIN 750 ML,1,2,1
9,4,SCHWEPPES 1 LT,1,5,4


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
46,(71032),(71031),56.149611,42.596257,36.787676,0.655172,735.103448,0.000769,2.897415
47,(71031),(71032),42.596257,56.149611,36.787676,0.863636,735.103448,0.000769,7.324718
64,(71549),(71550),48.404837,85.192513,27.106709,0.560000,314.160000,0.000565,2.268676
2,(66305),(69443),44.532450,131.661157,23.234322,0.521739,189.391304,0.000484,2.085149
1,(64423),(70667),23.234322,271.067088,19.361935,0.833333,146.928571,0.000402,5.965970
5,(66647),(70693),19.361935,578.921852,19.361935,1.000000,82.555184,0.000400,inf


In [10]:
item_name.count()

SKU        8239
KEYWORD    8239
dtype: int64